# How to change the clock frequency of SimpleSerial targets?

The target firmware used in almost all of our [Jupyter notebooks](https://github.com/newaetech/chipwhisperer-jupyter) uses the [SimpleSerial protocol](https://chipwhisperer.readthedocs.io/en/latest/simpleserial.html).

SimpleSerial works over UART, on the `TIO1` and `TIO2` pins of the 20-pin target connector.

Whether it's SimpleSerial [v1](https://chipwhisperer.readthedocs.io/en/latest/simpleserial.html#simpleserial-v1-1) or [v2](https://chipwhisperer.readthedocs.io/en/latest/simpleserial.html#simpleserial-v2-1), the target uses a fixed baud rate to communicate: 38400 bps for v1 and 230400 bps for v2. The target firmware establishes this in relation to its clock frequency, which is expected to be 7.37 MHz.

If you want to change the target clock, you have two options:

1. Modify and recompile the target firmware to account for your new desired clock frequency. This is unnecessarily complicated.
2. Scale the baud rate that is used to communicate with the target - no recompilation required!

The scaling is linear: consider for example a v1 target which expects to communicate at 38400 bps when clocked at 7.37 MHz. If the actual clock frequency is doubled to 14.74 MHz, then the rate that the target will communicate at also doubles to become 76800 bps.

**Supported Capture Hardware:**\
✅ CW-Nano\
✅ CW-Lite\
✅ CW-Pro\
✅ CW-Husky

**Required ChipWhisperer software installation:**\
✅ any release

In [1]:
%run '../connect.ipynb'
cw.scope_logger.setLevel(cw.logging.WARNING) # turn warnings back on

Let's confirm the default settings that `../connect.ipynb` gave us via `scope.default_setup()` and `cw.target()`:

In [2]:
CLOCK = scope.clock.clkgen_freq
BAUD = target.baud
print('Clock: %f MHz\nBaud: %d bps' % (CLOCK, BAUD))

Clock: 7363636.363636 MHz
Baud: 38400 bps


We can confirm that communication is working by running a capture.

If your target is running `simpleserial-aes` firmware, this will run the target AES operation. For different target firmware, substitute with what's needed to make your target "go":

In [3]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

Next we define our new clock frequency and change the baud rate accordingly:

In [4]:
NEWCLOCK = 12e6
scope.clock.clkgen_freq = NEWCLOCK
target.baud = BAUD * NEWCLOCK / CLOCK

Because we adjusted `target.baud`, the capture will still work at the new clock frequency:

In [5]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [6]:
assert not scope.adc.errors, scope.adc.errors